In [1]:
import re
import random
from collections import defaultdict
from itertools import chain
import pandas as pd
import codecs
import json
import os
from glob import glob

In [2]:
data_dir= "../data/events/"

In [73]:
df_ls=[]
for file in os.listdir(data_dir):
    if file.endswith(".csv"):
        filepath = os.path.join(data_dir, file)
        dataf = pd.read_csv(filepath)
        df_ls.append(dataf)

In [74]:
dataframes = pd.concat(df_ls)

In [78]:
columns = ['event', 'leading_event', 'date', 'category', 'entities']

In [79]:
df = dataframes[columns]

In [82]:
df = df.sort_values(by=["date"])

In [84]:
df.to_csv("../data/events/events.csv", index=False)

# Export events dataframes

In [9]:
df_ls = []
for file in glob("../data/output/events_dict_**.json"):
    date = file.split("/")[-1].replace(".json", "").split("_")[-1]
    with open(file) as reader:
        data = json.load(reader)
    df= pd.DataFrame.from_dict(data, orient="index")
    df_ls.append(df)

In [10]:
df_merge = pd.concat(df_ls)

In [15]:
df_merge.sort_values(by=["date", "cateogry", "main_event"],inplace=True)

In [17]:
df_merge.to_csv("../data/events/events_20220224-20220304.csv", index=False)

In [11]:
file = data_dir+"events_dict_2022-02-28.json"
with open(file) as reader:
    data = json.load(reader)

In [61]:
def event_df(data):
    
    eventls=[]
    leadings =[]
    dates = []
    categories =[]
    entities_ls =[]
    
    count=0
    for idx, event_dict in data.items():
        events=event_dict["events"]
        leading = event_dict["leading"]
        date=event_dict["date"]
        category = event_dict["category"]
        entities=event_dict["entities"]
        
        # event_entity_dict=defaultdict(list)
        
        for event in events:
            entities4event=[]
            
            categories.append(category)
            if len(entities)>0:
                for entity in entities:
                    if entity in event or event in entity:
                        entities4event.append(entity)
            count+=1
        
            if len(entities4event)>0:
                entities_ls.append(list(set(entities4event)))
            else:
                entities_ls.append([])
            
        leadings.append([leading for _ in range(len(events))])
        dates.append([date for _ in range(len(events))])
        eventls.append(events)
        
    events_ = list(chain.from_iterable(eventls))
    leadings_ = list(chain.from_iterable(leadings))
    dates_ = list(chain.from_iterable(dates))
    
    
    df= pd.DataFrame.from_dict({
        "event":events_,
        "leading_event":leadings_,
        "date":dates_,
        "category":categories,
        "entities":entities_ls
    })
    
    return df
    

In [3]:
test_file = "../data/output/events_dict_2022-03-2.json"
date = test_file.split("/")[-1].replace(".json", "").split("_")[-1]
print(date)
with open(test_file) as reader:
    data= json.load(reader)

2022-03-2


In [7]:
df= pd.DataFrame.from_dict(data, orient="index")

In [8]:
df

,cateogry,event_text,main_event,date,news,entities
0,Armed conflicts and attacks,The Ukrainian Air Force says that it has destr...,Russo-Ukrainian War,2022-03-2,[https://news.sky.com/story/ukraine-invasion-p...,"[Ukrainian Air Force, Russian Air Force, Dogfi..."
1,Armed conflicts and attacks,Russian Ground Forces say that they have captu...,Russo-Ukrainian War,2022-03-2,[https://edition.cnn.com/europe/live-news/ukra...,"[Russian Ground Forces, Black Sea, Kherson, Mi..."
2,Armed conflicts and attacks,The mayor of Kherson says that the city has fa...,Russo-Ukrainian War,2022-03-2,[https://www.nytimes.com/live/2022/03/02/world...,[]
3,Armed conflicts and attacks,The mayor of Mariupol reports mass casualties ...,Russo-Ukrainian War,2022-03-2,[https://www.reuters.com/world/mariupol-mayor-...,[Mariupol]
4,Armed conflicts and attacks,Ukrainian forces begin an offensive towards Ho...,Russo-Ukrainian War,2022-03-2,[https://www.ukrinform.net/rubric-ato/3417782-...,[Horlivka]
5,Armed conflicts and attacks,Two Kalibr cruise missiles strike a hospital i...,Russo-Ukrainian War,2022-03-2,[https://time.com/6153790/russia-ukraine-more-...,"[3M-54 Kalibr, Cruise missiles, Healthcare in ..."
6,Armed conflicts and attacks,Russian forces launch airstrikes on Kharkiv Na...,Russo-Ukrainian War,2022-03-2,[https://www.cnn.com/2022/03/02/europe/ukraine...,"[Airstrike, Kharkiv National University]"
7,Armed conflicts and attacks,Russia says that its troops have captured the ...,Russo-Ukrainian War,2022-03-2,[https://www.thedrive.com/the-war-zone/44534/r...,"[Zaporizhzhia Nuclear Power Plant, Nuclear pow..."
8,Armed conflicts and attacks,A missile strikes the Bangladeshi-flagged carg...,Russo-Ukrainian War,2022-03-2,[https://www.thedailystar.net/news/world/2022-...,"[Bangladesh, Mykolaiv Oblast]"
9,Armed conflicts and attacks,An emergency special session of the UN General...,Russo-Ukrainian War,2022-03-2,[https://www.washingtonpost.com/world/un-assem...,[Eleventh emergency special session of the Uni...


In [62]:
event_df(data)

,event,leading_event,date,category,entities
0,Russo-Ukrainian War,Russo-Ukrainian War,2022-02-28,Armed conflicts and attacks,[]
1,2022 Russian invasion of Ukraine,2022 Russian invasion of Ukraine,2022-02-28,Armed conflicts and attacks,"[Ukraine, 2022 Russian invasion of Ukraine, Be..."
2,Battle of Kharkiv,2022 Russian invasion of Ukraine,2022-02-28,Armed conflicts and attacks,"[Kharkiv, Battle of Kharkiv (2022)]"
3,Dozens of people are killed and hundreds more ...,2022 Russian invasion of Ukraine,2022-02-28,Armed conflicts and attacks,"[Kharkiv, MLRS]"
4,"Hennadiy Matsehora, the mayor of Kupiansk who ...",2022 Russian invasion of Ukraine,2022-02-28,Armed conflicts and attacks,"[Russia, Kupiansk]"
...,...,...,...,...,...
58,2022 Russian invasion of Ukraine,2022 Russian invasion of Ukraine,2022-02-28,Law and crime,"[2022 Russian invasion of Ukraine, Ukraine]"
59,Prosecutor Karim Ahmad Khan announces that the...,2022 Russian invasion of Ukraine,2022-02-28,Law and crime,"[Karim Ahmad Khan, Ukraine, International Crim..."
60,International reactions to the 2022 Russian in...,International reactions to the 2022 Russian in...,2022-02-28,Sports,[]
61,FIFA and UEFA suspend all Russian football clu...,FIFA and UEFA suspend all Russian football clu...,2022-02-28,Sports,"[2022 FIFA World Cup, FIFA, Qatar, Russian nat..."
